## Feed-Forward Neural Network

This file is used to run the optimally-determined model as per grid search in FFNN_Grid_Searcher

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
!pip install scikeras keras

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score

In [4]:
def create_model(first_layer=128, activation_1='relu', second_layer=64, activation_2='relu', activation_out='softmax', learn_rate=0.001):
    model = Sequential()
    model.add(Dense(first_layer, input_shape=(100,), activation=activation_1))
    model.add(Dense(second_layer, activation=activation_2))
    model.add(Dense(num_classes, activation=activation_out))
    model.compile(optimizer=Adam(learning_rate=learn_rate), loss=LOSS_TYPE, metrics=METRICS)
    return model

In [8]:
df = pd.read_csv('features.csv')
num_classes = 120

ACTIVATION_1 = 'relu'
ACTIVATION_2 = 'relu'
ACTIVATION_OUT = 'softmax'
TEST_SIZE = 0.25
EPOCHS = 20
METRICS = ['accuracy']
LOSS_TYPE = 'categorical_crossentropy'

In [12]:
x = df.drop(columns=['Breed'])
y = df['Breed']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# Encode the categorical target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the encoded labels to one-hot encoding
y_train = to_categorical(y_train_encoded, num_classes=num_classes)
y_test = to_categorical(y_test_encoded, num_classes=num_classes)

optimal = create_model(first_layer=128, activation_1='relu', second_layer=64, activation_2='relu', activation_out='softmax', learn_rate=0.0005)
optimal.fit(x_train, y_train, epochs=EPOCHS, batch_size=50, validation_data=(x_test, y_test))

# Make predictions
y_pred_opt = optimal.predict(x_test)

# Convert continuous predictions to class labels
y_pred_classes = np.argmax(y_pred_opt, axis=1)

# Convert one-hot encoded y_test back to class labels
y_test_classes = np.argmax(y_test, axis=1)

# Inverse transform the predicted and ground truth class labels to original breed names
y_pred_breed = label_encoder.inverse_transform(y_pred_classes)
y_test_breed = label_encoder.inverse_transform(y_test_classes)

# Evaluate the model
loss, accuracy = optimal.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy}')

multi_conf_matrix = multilabel_confusion_matrix(y_test_breed, y_pred_breed)
print("Confusion Matrix for each label : ")
print(multi_conf_matrix)

print("Classification Report : ")
print(classification_report(y_test_breed, y_pred_breed))

conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

Epoch 1/20
327/327 [==============================] - 1s 2ms/step - loss: 2.8517 - accuracy: 0.3802 - val_loss: 1.2323 - val_accuracy: 0.6711
Epoch 2/20
327/327 [==============================] - 0s 1ms/step - loss: 0.9036 - accuracy: 0.7442 - val_loss: 0.8698 - val_accuracy: 0.7452
Epoch 3/20
327/327 [==============================] - 0s 1ms/step - loss: 0.6753 - accuracy: 0.7946 - val_loss: 0.7893 - val_accuracy: 0.7614
Epoch 4/20
327/327 [==============================] - 0s 1ms/step - loss: 0.5798 - accuracy: 0.8218 - val_loss: 0.7593 - val_accuracy: 0.7653
Epoch 5/20
327/327 [==============================] - 0s 1ms/step - loss: 0.5219 - accuracy: 0.8348 - val_loss: 0.7318 - val_accuracy: 0.7770
Epoch 6/20
327/327 [==============================] - 0s 1ms/step - loss: 0.4755 - accuracy: 0.8468 - val_loss: 0.7182 - val_accuracy: 0.7876
Epoch 7/20
327/327 [==============================] - 0s 1ms/step - loss: 0.4407 - accuracy: 0.8585 - val_loss: 0.7303 - val_accuracy: 0.7795
Epoch 